<a href="https://www.kaggle.com/code/volkanatalay/ar-prediction-imbalanced-and-balanced-datasets?scriptVersionId=136325994" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 ## **Abstract** 

QSAR Androgen Receptor Data Set contains binder/positive (199) and non-binder/negative (1488) molecules (data instances). Therefore, the dataset is imbalanced (with a ratio of 7.5). Here, I use shallow binary classifiers and evaluate their performances on two configurations of the dataset:  (Configuration I) stratified 5-fold cross-validation (as mentioned in the original paper) and (Configuration II) balanced dataset obtained by using Butina clustering on the molecules. 

 ## **1. PROBLEM DEFINITION**

This is a binary classification problem: we need to predict whether a given molecule (data instance) is a binder to the Androgen Receptor or not. 

 ## **2. DATASET**

Each data instance corresponds to a molecule. A molecule is represented by molecular fingerprints consisting of 1024 bits. The last bit in each row is the target label: binder/positive or non-binder/negative.

In [ ]:
qsar_data = pd.read_csv('/kaggle/input/qsar-androgen-receptor-data-set/qsar_androgen_receptor.csv', header=None, sep=';')
print(qsar_data.shape)
qsar_data.head()

In [ ]:
import sklearn # learning algorithms
from sklearn.model_selection import train_test_split 
# use this to split input dataset into training dataset and independent test dataset 

 ## **3. <u>CONFIGURATION I</u>: STRATIFIED 5-FOLD CROSS-VALIDATION**

This is the configuration where a training, validation, and  test dataset splits are generated by keeping the same percentages of data instances of the two classes (binders/positive and non-binders/negative) in each split. However, in this case, since dataset is initially imbalanced, this imbalance between two classes will be kept in the splits as well.

A. Data is split into training dataset and independent test dataset. Independent test dataset is only used for tests while 5-fold cross-validation is applied with the training dataset.

B. Since the attributes (fingerprints) are in the first 1024 column of a row (data instance/molecule) and the last column indicates the label (positive or negative), these are separated and stored in different lists.

C. The labels indicated as positive or negative are then encoded by a bit (0 or 1).

D. Support Vector Machine (SVM) is employed as a shallow classifier. The hyperparameter values of the SVM are tuned by using 5-fold cross-validation and grid searh on a selected set of hyperparameters. 

E. The SVM-based classifier model is then evaluated in terms of a few metrics by using 5-fold cross-validation. 

F. An SVM-based classifier model is subsequently obtained using the tuned hyperparameter values and the whole training dataset. The classifier model is used to get predictions for the independent test dataset. The performance of the classifier model is evaluated both on the training dataset and on the independent test dataset.

In [ ]:
# A. Data is split into training dataset and independent test dataset

# last column of data indicates the desired output
desired_outputs = qsar_data.iloc[:,-1]

# split dataset into training 0.75 and test 0.25 parts
# stratified split for training dataset and test dataset according to the labels, that is "desired outputs"
training_dataset, test_dataset = train_test_split(qsar_data, test_size=0.25, stratify=desired_outputs)

In [ ]:
# B. Attributes and labels are separated and stored in different lists

# last column indicate the corresponding data instance (row) is positive or negative
# first 1024 columns are feature values ECFP4

# for training dataset
training_desired_outputs = training_dataset.iloc[:,-1]
training_input_features = training_dataset.iloc[:, 0:1024]

# for test dataset
test_desired_outputs = test_dataset.iloc[:,-1]
test_input_features = test_dataset.iloc[:, 0:1024]

In [ ]:
# C. Labels indicated as positive or negative are then encoded by a bit (0 or 1)

from sklearn import preprocessing

# encode positive and negative as 0 and 1
label_encoder = preprocessing.LabelEncoder()
training_desired_outputs = label_encoder.fit_transform(training_desired_outputs)
test_desired_outputs = label_encoder.fit_transform(test_desired_outputs)
#training_desired_outputs

In [ ]:
# D. Tune hyperparameter values of the SVM by using 5-fold cross-validation and grid search. 

# Evaluation metric values are displayed for each of the 5 folds. 

from sklearn import svm
from sklearn.model_selection import GridSearchCV

svc=svm.SVC()
# declare parameters for hyperparameter tuning
parameters = [ {'C':[1, 10, 100, 1000], 'kernel':['linear']},
               {'C':[1, 10, 100, 1000], 'kernel':['rbf'], 'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
               {'C':[1, 10, 100, 1000], 'kernel':['poly'], 'degree': [2,3,4] ,'gamma':[0.01,0.02,0.03,0.04,0.05]} 
              ]

grid_search = GridSearchCV(estimator = svc,  
                           param_grid = parameters,
                           scoring = 'recall',
                           cv = 5,
                           verbose=1)
grid_search.fit(training_input_features, training_desired_outputs)

print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))
# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (grid_search.best_params_))

In [ ]:
# E. Evaluate SVM-based classifier model by using 5-fold cross-validation.

from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import cross_val_score, cross_validate

# best parameters for SVM
linear_svc=svm.SVC(kernel='linear', C=1.0) 

scoring = ['precision', 'recall', 'f1', 'matthews_corrcoef']
scores = cross_validate(linear_svc, training_input_features, training_desired_outputs, cv=5, scoring=scoring)
print("precision ", scores['test_precision'])
print("recall ", scores['test_recall'])
print("f1 score", scores['test_f1'])
print("matthews corr coef ", scores['test_matthews_corrcoef'])

When we look at the precision, F1-score and MCC values, there are significant variations among these 5 folds and this is indeed not a good sign. These variations have been most probably caused by the imbalanced data.

In [ ]:
# F. Obtain SVM-based classifier model using the tuned hyperparameter values and the whole training dataset. 
# Performance of the classifier model is evaluated both on the training dataset and on the independent test dataset.


from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt


# svm trained by the whole training set
linear_svc.fit(training_input_features, training_desired_outputs)

# predictions for the training set
training_predicted_outputs = linear_svc.predict(training_input_features)
# predictions for the independent test set 
test_predicted_outputs = linear_svc.predict(test_input_features)

print("performance metrics for training")
print(classification_report(training_desired_outputs, training_predicted_outputs))
print()
print()

# performance metrics for test    
print("performance metrics for test")
print(classification_report(test_desired_outputs, test_predicted_outputs))
# confusion matrix for test set
conf_matrix = confusion_matrix(test_desired_outputs, test_predicted_outputs)
ConfusionMatrixDisplay(conf_matrix).plot(cmap='GnBu')    

Although the overall test results seem to be fine, there is a problem with Class 1 (as can be observed from the confusion matrix): 29 (out of 50) instances of Class 1 are predicted to be Class 0. This is most probably caused by the imbalance in the training dataset. 

 ## **4. <u>CONFIGURATION II</u>: BALANCED DATASET BY USING BUTINA CLUSTERING**

This is the configuration where a training, validation, and  test dataset splits are generated by keeping the <u>same number of data instances</u> of the two classes (binders/positive and non-binders/negative) in each split. In this case, since dataset is initially balanced, this balance between two classes will be kept in the splits as well.

A. rdkit package is used for Butina clustering algorithm. rdkit and some other packages are imported.

B. Butina clustering algorithm is implemented as a function that inputs directly the fingerprints of a molecule. Butina clustering algorithm uses Tanimoto similarity between two fingerprints.

C. The binders/posiitive and non-binders/negative compounds are grouped separately. 

D. Preprocessing is applied both on binders/posiitive and non-binders/negative data instances.

E. Butina clustering algorithm is applied separately on the posiitive instance group and negative instance group. 
The labels indicated as positive or negative are then encoded by a bit (0 or 1).

F. Cluster centers and their corresponding molecules' figerprints are stored.

G. Data instances belonging to positive set is split into training dataset and test dataset. Same procedure is applied for the data instances belonging to negative set. From these, final training dataset ans final test dataset is formed.

H. Output labels (0 and 1) are associated.

I. The SVM-based classifier model is evaluated in terms of a few metrics by using 5-fold cross-validation. 

J. The SVM-based classifier model is subsequently obtained using the whole training dataset. The classifier model is used to get predictions for the independent test dataset. The performance of the classifier model is evaluated both on the training dataset and on the independent test dataset.

In [ ]:
# A. rdkit and some other packages are imported
import sys
!{sys.executable} -m pip install rdkit

import rdkit
import pandas as pd

from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools, Draw

In [ ]:
# B. Butina clustering algorithm is implemented as a function

# Butina clustering function that input directly the fingerprints
def butina_cluster(fp_list,cutoff=0.35):
    dists = []
    nfps = len(fp_list)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fp_list[i],fp_list[:i])
        dists.extend([1-x for x in sims])
    mol_clusters = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
   
    cluster_id_list = [0]*nfps
    for idx,cluster in enumerate(mol_clusters,1):
        for member in cluster:
            cluster_id_list[member] = idx
            
    return mol_clusters, cluster_id_list

In [ ]:
# C. The binders/posiitive and non-binders/negative compounds are grouped separately. 


# group rows in terms of being positive or negative
grouped = qsar_data.groupby(qsar_data.iloc[:,-1])
df_positive = grouped.get_group("positive")
df_negative = grouped.get_group("negative")

In [ ]:
# D. Preprocessing is applied both on binders/posiitive and non-binders/negative data instances.

# convert positive group into ExplicitBitVect
df_pos_fp_int = df_positive.iloc[:, 0:1024]

# convert dataframe into a list of ExplicitBitVect
pos_fp_int_list = df_pos_fp_int.values.tolist() # these should be converted into ExplicitBitVect
pos_fp_list =[]
for mol in pos_fp_int_list:
    temp_str = ''.join(str(x) for x in mol)
    pos_fp_list.append(DataStructs.CreateFromBitString(temp_str))
    
# convert negative group into ExplicitBitVect
df_neg_fp_int = df_negative.iloc[:, 0:1024]

# convert dataframe into a list of ExplicitBitVect
neg_fp_int_list = df_neg_fp_int.values.tolist() # these should be converted into ExplicitBitVect
neg_fp_list =[]
for mol in neg_fp_int_list:
    temp_str = ''.join(str(x) for x in mol)
    neg_fp_list.append(DataStructs.CreateFromBitString(temp_str))
    
print("number of positives ", len(pos_fp_list))
print("number of negatives ", len(neg_fp_list))

In [ ]:
# E. Butina clustering algorithm is applied separately on the posiitive instance group and negative instance group. 

# cluster the negative compounds
neg_clusters, neg_cluster_idList = butina_cluster(neg_fp_list, cutoff=0.5)
# cluster the positive compounds
pos_clusters, pos_cluster_idList = butina_cluster(pos_fp_list, cutoff=0.5)

In [ ]:

# F. Cluster centers and their sorresponding molecules' figerprints are stored.

# Get the cluster center of each cluster (first molecule in each cluster)
pos_cluster_centers = [c[0] for c in pos_clusters]
# How many cluster centers/clusters do we have?
print("Number of cluster centers:", len(pos_cluster_centers))

# Get the cluster center of each cluster (first molecule in each cluster)
neg_cluster_centers = [c[0] for c in neg_clusters]
# How many cluster centers/clusters do we have?
print("Number of cluster centers:", len(neg_cluster_centers))

# data from pos_fp_int_list for pos cluster centers
# data from neg_fp_int_list for neg cluster centers
pos_molecules = [ pos_fp_int_list[m] for m in pos_cluster_centers]
neg_molecules = [ neg_fp_int_list[m] for m in neg_cluster_centers]

In [ ]:

# G. Training dataset and test dataset splits are obtained. 
#    From these, final training dataset ans final test dataset is formed.

import sklearn
from sklearn.model_selection import train_test_split


pos_training_dataset, pos_test_dataset = train_test_split(pos_molecules, test_size=0.25, random_state=43)
neg_training_dataset, neg_test_dataset = train_test_split(neg_molecules, train_size=len(pos_training_dataset), random_state=43)

print("pos training test", len(pos_training_dataset), len(pos_test_dataset))
print("neg training test", len(neg_training_dataset), len(neg_test_dataset))

training_dataset = pos_training_dataset + neg_training_dataset
test_dataset = pos_test_dataset + neg_test_dataset

In [ ]:
# H. Generate output lists positive 1, negative 0

training_output = [1] * len(pos_training_dataset)
temp = [0] * len(neg_training_dataset)
training_output += temp

test_output = [1] * len(pos_test_dataset)
temp = [0] * len(neg_test_dataset)
test_output += temp

In [ ]:
# I. Evaluate SVM-based classifier model by using 5-fold cross-validation.

from sklearn import svm
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import cross_val_score, cross_validate

# best parameters for SVM
clf=svm.SVC(kernel='linear', C=10) 

scoring = ['precision', 'recall', 'f1', 'matthews_corrcoef']
scores = cross_validate(clf, training_dataset, training_output, cv=5, scoring=scoring)
print("precision ", scores['test_precision'])
print("recall ", scores['test_recall'])
print("f1 score", scores['test_f1'])
print("matthews corr coef ", scores['test_matthews_corrcoef'])

In [ ]:
# J. An SVM-based classifier model is subsequently obtained using the whole training dataset. 

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report


# support vector classifier
 
clf.fit(training_dataset, training_output)


# predictions for training dataset
training_predicted_outputs = clf.predict(training_dataset)
# predictions for test dataset
test_predicted_outputs = clf.predict(test_dataset)

print("for training")
print(classification_report(training_output, training_predicted_outputs))
 
print("for test")
print(classification_report(test_output, test_predicted_outputs))

conf_matrix = confusion_matrix(test_output, test_predicted_outputs)
ConfusionMatrixDisplay(conf_matrix).plot(cmap='GnBu')   
